In [1]:
from lxml import etree as ET

In [2]:
xml_data = ET.parse("darv_or.xml")

In [3]:
root = xml_data.getroot()

In [4]:
xmlstr = ET.tostring(root, encoding='utf8', method='xml')

In [5]:
or_namespaces = {'are': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_answer_or/v_1.0.2', 
              'dtt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_datatypes/v_1.0.2',
              'udt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/uvis_datatypes/v_1.0.1',
              'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
             }

In [6]:
inverse_or_namespaces = {v: k for k, v in or_namespaces.items()}

In [7]:
nadpis = xml_data.xpath('/are:Ares_odpovedi/are:Odpoved/dtt:Vypis_OR/dtt:Zakladni_udaje/dtt:Obchodni_firma',namespaces = or_namespaces)
for of in nadpis:
    print(of.text,of.get("ddo"))
    

print(nadpis[0].attrib)

MADETA a. s. None
MADETA Group a. s. 2006-06-30
JIHOMILK, akciová společnost 2005-07-04
{'dod': '2006-06-30'}


## Profiling xml 

In [82]:
from lxml import etree as ElementTree
import pandas as pd

# Nahrazení namespaců, aby bylo lépe čitelné

or_namespaces = {'are': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_answer_or/v_1.0.2', 
              'dtt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/ares/ares_datatypes/v_1.0.2',
              'udt': 'http://wwwinfo.mfcr.cz/ares/xml_doc/schemas/uvis_datatypes/v_1.0.1',
              'xsi': 'http://www.w3.org/2001/XMLSchema-instance'
             }
inverse_or_namespaces = {v: k for k, v in or_namespaces.items()}
def replace_ns(path_w_ns):
    for key, name in inverse_or_namespaces.items():
        path_w_ns = path_w_ns.replace(key, name)
    path_wo_ns = path_w_ns
    path_wo_ns = path_wo_ns.replace('{','')
    path_wo_ns = path_wo_ns.replace('}',':')
    return path_wo_ns

# Elementy XML a jejich xpath cesty
def etree_iter_path(node, tag=None, path='.'):
    if tag == "*":
        tag = None
    if tag is None or node.tag == tag:
        yield node, path
    for child in node:
        _child_path = '%s/%s' % (path, child.tag)
        for child, child_path in etree_iter_path(child, tag, path=_child_path):
            yield child, child_path

# Dotažení info o XML do pd.DataFrame
xmldoc = ElementTree.parse("darv_or.xml")
list_data = []
for elem, path in etree_iter_path(xmldoc.getroot()):
    elem_parent = ''
    if elem.getparent() is not None:
        elem_parent = replace_ns(elem.getparent().tag)
    attribu = elem.attrib
    attr_keys_string = ",".join(attribu.keys())
    list_data.append([replace_ns(elem.tag), elem_parent,attr_keys_string,replace_ns(path)])
pd_data = pd.DataFrame(list_data)
pd_data.head(5)

,0,1,2,3
0,are:Ares_odpovedi,,"odpoved_datum_cas,odpoved_pocet,odpoved_typ,vy...",.
1,are:Odpoved,are:Ares_odpovedi,,./are:Odpoved
2,dtt:Pomocne_ID,are:Odpoved,,./are:Odpoved/dtt:Pomocne_ID
3,dtt:Vysledek_hledani,are:Odpoved,,./are:Odpoved/dtt:Vysledek_hledani
4,dtt:Kod,dtt:Vysledek_hledani,,./are:Odpoved/dtt:Vysledek_hledani/dtt:Kod


In [83]:
import pandas_profiling
pandas_profiling.ProfileReport(pd_data)

UnboundLocalError: local variable 'pearson_matrix' referenced before assignment

In [86]:
pd_no_dupl = pd_data.drop_duplicates()

In [87]:

"""Zapsání """
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('xml_profile.xlsx')

# Convert the dataframe to an XlsxWriter Excel object.
pd_no_dupl.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [35]:
"""pd_distinct_data = pd_data.drop_duplicates()"""
pd_data.head(33)

,a,b,c,d
0,are:Ares_odpovedi,"[odpoved_datum_cas, odpoved_pocet, odpoved_typ...","2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",.
1,are:Odpoved,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved
2,dtt:Pomocne_ID,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Pomocne_ID
3,dtt:Vysledek_hledani,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Vysledek_hledani
4,dtt:Kod,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Vysledek_hledani/dtt:Kod
5,dtt:Pocet_zaznamu,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Pocet_zaznamu
6,dtt:Vypis_OR,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Vypis_OR
7,dtt:Uvod,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Vypis_OR/dtt:Uvod
8,dtt:Nadpis,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Vypis_OR/dtt:Uvod/dtt:Nadpis
9,dtt:Aktualizace_DB,[],"2018-08-23T09:19:43,1,Vypis_OR,XML,klient,http...",./are:Odpoved/dtt:Vypis_OR/dtt:Uvod/dtt:Aktual...
